# Deploy and perform inference on Model Package from AWS Marketplace 

This notebook provides you instructions on how to deploy and perform inference on model packages from AWS Marketplace object detection model.

This notebook is compatible only with those object detection model packages which this notebook is linked to.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to this object detection model. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

**Note** - This notebook requires you to follow instructions and specify values for parameters, as instructed.

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page you opened this notebook for.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn='<Customer to specify Model package ARN corresponding to their AWS region>' 

In [ ]:
import json 
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import matplotlib.patches as patches
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageColor

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()
boto3 = sagemaker_session.boto_session
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name

s3 = boto3.client("s3")
runtime= boto3.client('runtime.sagemaker')

In next step, you would be deploying the model for real-time inference. For  information on how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### 2. Create an endpoint and perform real-time inference

In [ ]:
model_name='object-detection-model'

In [ ]:
#The object detection model packages this notebook notebook is compatible with, support application/x-image as the 
#content-type.
content_type='application/x-image'

Review and update the compatible instance type for the model package in the following cell.

In [ ]:
real_time_inference_instance_type='ml.g4dn.xlarge'
batch_transform_inference_instance_type='ml.p2.xlarge'

#### A. Create an endpoint

In [ ]:
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Prepare input file for performing real-time inference
In this step, we will download class_id_to_label_mapping from S3 bucket. The mapping files has been downloaded from [TensorFlow](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0).

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "inference-notebook-assets"

def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

img_jpg = "Naxos_Taverna.jpg"

#Download image
download_from_s3(key_filenames=[img_jpg])

In [ ]:
#Mapping from model predictions to class labels 
class_id_to_label = {"1": "person", "2": "bicycle", "3": "car", "4": "motorcycle", "5": "airplane", "6": "bus", "7": "train", "8": "truck", "9": "boat", "10": "traffic light", "11": "fire hydrant", "13": "stop sign", "14": "parking meter", "15": "bench", "16": "bird", "17": "cat", "18": "dog", "19": "horse", "20": "sheep", "21": "cow", "22": "elephant", "23": "bear", "24": "zebra", "25": "giraffe", "27": "backpack", "28": "umbrella", "31": "handbag", "32": "tie", "33": "suitcase", "34": "frisbee", "35": "skis", "36": "snowboard", "37": "sports ball", "38": "kite", "39": "baseball bat", "40": "baseball glove", "41": "skateboard", "42": "surfboard", "43": "tennis racket", "44": "bottle", "46": "wine glass", "47": "cup", "48": "fork", "49": "knife", "50": "spoon", "51": "bowl", "52": "banana", "53": "apple", "54": "sandwich", "55": "orange", "56": "broccoli", "57": "carrot", "58": "hot dog", "59": "pizza", "60": "donut", "61": "cake", "62": "chair", "63": "couch", "64": "potted plant", "65": "bed", "67": "dining table", "70": "toilet", "72": "tv", "73": "laptop", "74": "mouse", "75": "remote", "76": "keyboard", "77": "cell phone", "78": "microwave", "79": "oven", "80": "toaster", "81": "sink", "82": "refrigerator", "84": "book", "85": "clock", "86": "vase", "87": "scissors", "88": "teddy bear", "89": "hair drier", "90": "toothbrush"}

#### C. Query endpoint that you have created with the opened images

In [ ]:
#perform_inference method performs inference on the endpoint and prints predictions.
def perform_inference(filename):
    response = runtime.invoke_endpoint(EndpointName='test-tensorflow-test', ContentType=content_type, Body=input_img)
    model_predictions = json.loads(response['Body'].read())
    return model_predictions

In [ ]:
with open(img_jpg, 'rb') as file: input_img = file.read()
model_predictions = perform_inference(input_img)
result = {key: np.array(value)[np.newaxis, ...] if isinstance(value, list) else np.array([value]) for key, value in model_predictions['predictions'][0].items()}

#### D. Display model predictions as bounding boxes on the input image 

In [ ]:
colors = list(ImageColor.colormap.values())

image_pil = Image.open(img_jpg)
image_np = np.array(image_pil)

plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
classes = [class_id_to_label[str(int(index))] for index in result["detection_classes"][0]]
bboxes, confidences = result["detection_boxes"][0], result["detection_scores"][0]
for idx in range(20):
    if confidences[idx] < 0.3:
        break
    ymin, xmin, ymax, xmax = bboxes[idx]
    im_width, im_height = image_pil.size
    left, right, top, bottom = xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height
    x, y = left, bottom
    color = colors[hash(classes[idx]) % len(colors)]
    rect = patches.Rectangle((left, bottom), right-left, top-bottom, linewidth=3, edgecolor=color, facecolor='none')
    ax.add_patch(rect)
    ax.text(left, top, "{} {:.0f}%".format(classes[idx], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))

#### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_key_prefix = 'object-detection-model-transform-input'
transform_input = sagemaker_session.upload_data(img_jpg, key_prefix=transform_input_key_prefix) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

### 4. Clean-up

#### A. Delete the model

In [ ]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

